In [1]:
import sys
sys.path.append('../')
from utility.file_utility import FileUtility
import pandas as pd
import numpy as np
from data_access.data_access_utility import ABRDataAccess



In [8]:
feature_lists=[['snps_nonsyn_trimmed'],['gpa_trimmed','gpa_roary'],['genexp'],['snps_nonsyn_trimmed','gpa_trimmed','gpa_roary'],['genexp','gpa_trimmed','gpa_roary'],['genexp','snps_nonsyn_trimmed'],['snps_nonsyn_trimmed','gpa_trimmed','gpa_roary','genexp']]

test_res=dict()
for setting in ['block','standard']:
    for feature_list in feature_lists:
        ABRAccess=ABRDataAccess('../../less_important/amr_results/res_before_5_3_2017/intermediate_reps/',feature_list)
        for drug in ABRAccess.BasicDataObj.drugs:
            res_name=drug+'_'.join(feature_list)
            X_rep, Y, features, final_isolates = ABRAccess.get_xy_prediction_mats(drug, mapping={'0':0,'0.0':0,'1':1,'1.0':1})
            test_isolate=FileUtility.load_list('../temp_configs/cv3/'+setting+'_cv/'+drug+'_S-vs-R_test.txt')
            idx=[final_isolates.index(x) for x in test_isolate]
            X=X_rep[idx,:]
            Y_gt=[Y[i] for i in idx]
            path='/mounts/data/proj/asgari/dissertation/git_repos/pseudoresults/classifications_'+setting+'/'
            files=[path+res_name+'_LR.pickle']
            files+=[path+res_name+'_SVM.pickle']
            for file in files:
                try:
                    [label_set, conf, best_score_, best_estimator_,cv_results_, best_params_, pred]=FileUtility.load_obj(file)
                except:
                    [label_set, best_score_, best_estimator_,cv_results_, best_params_, pred]=FileUtility.load_obj(file)
                y_pred=best_estimator_.predict(X)
                test_res[file]=(Y_gt,y_pred)

Data access created..
@@@../../less_important/amr_results/res_before_5_3_2017/intermediate_reps/snps_nonsyn_trimmed_feature_vect.npz
[364, 238, 349, 58, 343, 196, 348, 97, 156, 50, 36, 40, 17, 299, 245, 360, 52, 385, 5, 312, 220, 373, 388, 102, 316, 96, 88, 383, 89, 342, 46, 114, 205, 219, 165, 43, 54, 288, 113, 105, 120, 363, 133, 162, 396, 160, 30, 257, 350, 246, 335, 199, 314, 248, 51, 72, 172, 304, 331, 390, 263, 254, 20, 112, 116, 264, 141, 236, 126, 139, 197, 344, 321, 243, 49, 137, 0, 252, 85, 291, 275]
/mounts/data/proj/asgari/dissertation/git_repos/pseudoresults/classifications_block/Tobramycinsnps_nonsyn_trimmed_LR.pickle
/mounts/data/proj/asgari/dissertation/git_repos/pseudoresults/classifications_block/Tobramycinsnps_nonsyn_trimmed_SVM.pickle


/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LinearSVC from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


[189, 109, 147, 102, 163, 245, 157, 38, 142, 134, 240, 59, 54, 265, 37, 27, 331, 25, 143, 276, 216, 263, 295, 197, 231, 217, 176, 258, 243, 162, 315, 118, 114, 70, 88, 120, 22, 62, 325, 167, 13, 110, 196, 133, 270, 229, 136, 173, 30, 159, 151, 131, 50, 199, 52, 326, 24, 203, 104, 48, 79, 181, 127, 180, 185, 63]
/mounts/data/proj/asgari/dissertation/git_repos/pseudoresults/classifications_block/Ceftazidimsnps_nonsyn_trimmed_LR.pickle
/mounts/data/proj/asgari/dissertation/git_repos/pseudoresults/classifications_block/Ceftazidimsnps_nonsyn_trimmed_SVM.pickle
[349, 50, 273, 185, 40, 157, 86, 332, 4, 184, 27, 218, 272, 350, 168, 303, 313, 210, 100, 162, 29, 194, 111, 275, 268, 19, 14, 232, 216, 108, 44, 2, 48, 159, 120, 342, 339, 231, 251, 63, 242, 285, 227, 163, 289, 84, 244, 151, 33, 15, 199, 224, 149, 61, 39, 53, 148, 7, 226, 323, 336, 318, 8, 109, 41, 319, 341, 276, 154, 177, 271]
/mounts/data/proj/asgari/dissertation/git_repos/pseudoresults/classifications_block/Ciprofloxacinsnps_nonsy

In [14]:
def get_cv_res(filename):
    try:
        [label_set, conf, best_score_, best_estimator_,cv_results_, best_params_, pred]=FileUtility.load_obj(filename)
    except:
        [label_set, best_score_, best_estimator_,cv_results_, best_params_, pred]=FileUtility.load_obj(filename)
    res=dict()
    idx=np.argmax(cv_results_['mean_test_f1_macro'])
    res['f1_macro*']=np.round(cv_results_['mean_test_f1_macro'][idx],2)
    res['f1_macro']=str(np.round(cv_results_['mean_test_f1_macro'][idx],2))+ ' ± ' + str(np.round(cv_results_['std_test_f1_macro'][idx],2))
    res['f1_micro']=str(np.round(cv_results_['mean_test_f1_micro'][idx],2))+ ' ± ' + str(np.round(cv_results_['std_test_f1_micro'][idx],2))
    res['precision_micro']=str(np.round(cv_results_['mean_test_precision_micro'][idx],2))+ ' ± ' + str(np.round(cv_results_['std_test_precision_micro'][idx],2))
    res['precision_macro']=str(np.round(cv_results_['mean_test_precision_macro'][idx],2))+ ' ± ' + str(np.round(cv_results_['std_test_precision_macro'][idx],2))
    res['recall_micro']=str(np.round(cv_results_['mean_test_recall_micro'][idx],2))+ ' ± ' + str(np.round(cv_results_['std_test_recall_micro'][idx],2))
    res['recall_macro']=str(np.round(cv_results_['mean_test_recall_macro'][idx],2))+ ' ± ' + str(np.round(cv_results_['std_test_recall_macro'][idx],2))
    res['accuracy']=str(np.round(cv_results_['mean_test_accuracy'][idx],2))+ ' ± ' + str(np.round(cv_results_['std_test_accuracy'][idx],2))
    res['tnr']=str(np.round(cv_results_['mean_test_tnr'][idx],2))+ ' ± ' + str(np.round(cv_results_['std_test_tnr'][idx],2))
    res['scores_p_0']=str(np.round(cv_results_['mean_test_scores_p_0'][idx],2))+ ' ± ' + str(np.round(cv_results_['std_test_scores_p_0'][idx],2))
    res['scores_r_0']=str(np.round(cv_results_['mean_test_scores_r_0'][idx],2))+ ' ± ' + str(np.round(cv_results_['std_test_scores_r_0'][idx],2))
    res['scores_f_1_0']=str(np.round(cv_results_['mean_test_scores_f_1_0'][idx],2))+ ' ± ' + str(np.round(cv_results_['std_test_scores_f_1_0'][idx],2))
    res['scores_f_1_1']=str(np.round(cv_results_['mean_test_scores_f_1_1'][idx],2))+ ' ± ' + str(np.round(cv_results_['std_test_scores_f_1_1'][idx],2))
    res['scores_p_1']=str(np.round(cv_results_['mean_test_scores_p_1'][idx],2))+ ' ± ' + str(np.round(cv_results_['std_test_scores_p_1'][idx],2))
    res['scores_r_1']=str(np.round(cv_results_['mean_test_scores_r_1'][idx],2))+ ' ± ' + str(np.round(cv_results_['std_test_scores_r_1'][idx],2)) 
    res['file']=filename
    return res

In [23]:
from sklearn.metrics.classification import precision_recall_fscore_support

def get_table_from_files(files, test_res):
    table = {'classifier': [], 'drug': [], 'feature':[],'test_macro_f1':[],'f1_macro':[],'f1_micro':[],'precision_micro':[],'precision_macro':[],'recall_micro':[],'recall_macro':[],'accuracy':[],'tnr':[],'scores_p_0':[],'scores_r_0':[],'scores_f_1_0':[],'scores_p_1':[],'scores_r_1':[],'scores_f_1_1':[]}
    drugs=['Ceftazidim','Tobramycin','Meropenem','Colistin','Ciprofloxacin']
    for file in files:        
        drug_feature_classifier=file.split('/')[-1].replace('.pickle','').split('_')
        
        
        if file in test_res:
            y_true, y_pred = list(test_res[file])
            p, r, f, _ = precision_recall_fscore_support(y_true, y_pred,
                                                     beta=1,
                                                     labels=None,
                                                     pos_label=1,
                                                     average='macro',
                                                     warn_for=('f-score'))
        test_macro_f1='-' if file not in test_res else f
        for drug in drugs:
            if drug in drug_feature_classifier[0]:
                drug_feature_classifier.insert(0,drug)
                drug_feature_classifier[1]=drug_feature_classifier[1].replace(drug,'')
                drug_feature_classifier=[drug_feature_classifier[0],'_'.join(drug_feature_classifier[1:-1]),drug_feature_classifier[-1]]
                res=get_cv_res(file)
                for key in table:
                    if key=='classifier':
                        table['classifier'].append(drug_feature_classifier[-1])
                    elif key=='drug':
                        table['drug'].append(drug_feature_classifier[0])
                    elif key=='feature':
                        table['feature'].append(drug_feature_classifier[1])
                    elif not key =='test_macro_f1':
                        table[key].append(res[key])
        table['test_macro_f1'].append(test_macro_f1)
    return pd.DataFrame(data=table,columns=['drug', 'feature','classifier','f1_macro','test_macro_f1','tnr','accuracy','f1_micro','precision_micro','precision_macro','recall_micro','recall_macro','scores_p_0','scores_r_0','scores_f_1_0','scores_p_1','scores_r_1','scores_f_1_1'])

In [24]:
path2res='/mounts/data/proj/asgari/dissertation/git_repos/pseudoresults/classifications_block/'
files_block=FileUtility.recursive_glob(path2res,'*.pickle')
path2res='/mounts/data/proj/asgari/dissertation/git_repos/pseudoresults/classifications_standard/'
files_std=FileUtility.recursive_glob(path2res,'*.pickle')

In [25]:
writer = pd.ExcelWriter('classification_31_5_18.xlsx')
get_table_from_files(files_block,test_res).to_excel(writer,'block')
get_table_from_files(files_std,test_res).to_excel(writer,'standard')
writer.save()

/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LinearSVC from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/mounts/Users/student/asgari/.local/lib/python3.4/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from v